# Implementing a Transformer from Scratch

Using PyTorch and basic libraries.

Based on this YouTube tutorial: https://www.youtube.com/watch?v=U0s0f995w14

Which is based on this paper: Attention is All You Need: https://arxiv.org/abs/1706.03762

Just a way for me to practice and understand the structure of Transformers.

In [3]:
#Imports

import torch
import torch.nn as nn

In [2]:
#@title MyTransformer Code


#Self Attention

class SelfAttention(nn.Module):

  def __init__(self, embed_size, heads): #heads is the number of heads over which the embedded input is distributed.

    super(SelfAttention, self).__init__()
    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = embed_size // heads

    #Checking to make sure the embed_size and heads are compatible with each other.
    assert (self.head_dim * heads == embed_size), "Embedding size needs to be divisible by heads"

    #Setting up Linear Layers

    #nn.Linear basically applies a simple linear transform. Bias = False removes the constant term.
    self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)

    #This is the fully connected output. Remember, self.head_dim * heads == embed_size
    self.fc_out = nn.Linear(heads*self.head_dim, embed_size)


  def forward(self, values, keys, query, mask):

    N = query.shape[0]

    #These will correspond to source sentence length and target, but bc this entire thing is an embedded class, we don't know where it's going to be used.
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1] 

    #Putting them into matrix form? Also splits them up based on the number of heads ig.
    values = values.reshape(N, value_len, self.heads, self.head_dim)
    keys = keys.reshape(N, key_len, self.heads, self.head_dim)
    queries = query.reshape(N, query_len, self.heads, self.head_dim)

    #Sending the above through the linear layers

    values = self.values(values)
    keys = self.keys(keys)
    queries = self.queries(queries)


    #Think this is just a function that does matrix multiplication, but on the specific dimensions you need. Pay attention to the string, it's dimensions.
    energy = torch.einsum('nqhd,nkhd->nhqk', [queries, keys])

    #queries shape: (N, query_len, heads, heads_dim)
    #keys shape: (N, key_len, heads, head_dim)
    #energy shape: (N, heads, query_len, key_len)

    #If mask is 0, then set the corresponding element to -inf. Mask means we don't want that element to influence the next step. 
    #if mask is not None:
    #  energy = energy.masked_fill(mask == 0, float("-1e20"))

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float("-1e20"))


    attention = torch.softmax(energy / self.embed_size ** (1/2), dim=3) #dim 3 means we are doing this for each key, which corresponds to a word in an input sentence basically.

    out = torch.einsum('nhql,nlhd->nqhd', [attention, values]).reshape(
        N, query_len, self.heads * self.head_dim
    )
    #attention (energy) shape : N, heads, query_len, key_len
    #values shape : N, value_len, heads, head_dim

    #we want to multiply on key_len and value_len bc these are corresponding to each other. Each key has a value.

    # -> out shape : N, query_len, heads, head_dim : Then we flatten for output.

    output = self.fc_out(out) #Passes through another linear layer
    return output


#Transformer Block

class TransformerBlock(nn.Module):

    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock,self).__init__()
        self.attention = SelfAttention(embed_size, heads) #Using the above SelfAttention
        
        #Normalization
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size) #as opposed to Batch Normalization. Read More!


        #This is pretty straightforward. Will be used as feed_forward in the block. Reference diagram for more details.
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, value, key, query, mask):

        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query)) # adding query creates the skip connection
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))

        return out
class Encoder(nn.Module):

    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length, #Of inputs
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion = forward_expansion
                )#Pretty sure there should be a num_layers list comprehension expression here. Think you missed this out, check back.
            for _ in range(num_layers)]
        )

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask): #x is input?

        N, seq_length = x.shape
        positions = torch.arange(0,seq_length).expand(N, seq_length).to(self.device) # This just puts positional encoding as 1,2,3,...seq_length. expand i think maps it onto each word?

        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions)) #We just send positions in, but as it learns, the positions values changes to the right one...?

        for layer in self.layers:

            out = layer(out,out,out, mask) #this is strange, but in the encoder, value key and query are all the same in this special case. Strange how this layers thing works.

        return out
class DecoderBlock(nn.Module):

    def __init__(self, embed_size, heads, dropout, forward_expansion, device):

        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)

        self.transformer_block = TransformerBlock(embed_size, heads, dropout=dropout, forward_expansion=forward_expansion)
        self.dropout = nn.Dropout(dropout)


    #here, src_mask is optional. When we pad the source, we want to mask out the padded bits to avoid doing any computation on them, bc they're supposed to be ignored.
    #trg_mask is important, i'm guessing bc it prevents the algorithm from peaking into the thing its supposed to predict.
    #notice how there's no query. In the decoder it's calculated and not given.
    def forward(self, x, value, key, src_mask, trg_mask):

        attention = self.attention(x,x,x, trg_mask) #Not entirely sure what this is. 
        query = self.dropout(self.norm(attention + x)) #adding x again creates a skip connection
        out = self.transformer_block(value, key, query, src_mask) #this is where the magic happens?

        return out
class Decoder(nn.Module):

    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [DecoderBlock(embed_size, heads, dropout=dropout, forward_expansion=forward_expansion, device=device) for _ in range(num_layers)]
        )
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
    
    def forward(self, x, enc_out, src_mask, trg_mask):

        N, seq_length = x.shape
        positions = torch.arange(0,seq_length).expand(N, seq_length).to(self.device)

        x = self.dropout(self.word_embedding(x)+self.position_embedding(positions))

        #Here, the encoder output comes in with value and key. the query will come from the rest of the decoder.
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
        
        out = self.fc_out(x)

        return out #He didnt mention this in the video, recheck
class MyTransformer(nn.Module):

    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=256,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cuda", #check this out when using colab
        max_length=100,
    ):
        super(MyTransformer, self).__init__()

    #Both of these are just using what we defined above. Why are all the hyperparams the same though? Why do they need to share the same dropout, num_layers, forward_expansion

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    #We need functions to make source and target masks. 

    def make_src_mask(self, src):

        #This is new to you. so if src is not equal to the pad token, make it 1 in the mask. If it's a pad token make it 0. This way the pad tokens are ignored in the mask
        #unsure about how this unqueeze thing works. Dimensions of output might be what is affected.
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2) 
        # (N, 1,1, src_len)

        return src_mask.to(self.device)

    def make_trg_mask(self, trg): #trg is the input? unsure 
        N, trg_len = trg.shape

        #Remember, we basically want a lower triangular matrix filled with 1s. This is so the decoder can only see past info, not peak ahead. 
        #Unsure about expand command
        #the tricks to making that matrix:
        trg_mask = torch.tril(torch.ones((trg_len,trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [3]:
#This is copy pasted from the github repo, just for the tensor examples.

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = MyTransformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)
    out = model(x, trg[:, :-1])
    print(out.shape)

cpu
torch.Size([2, 7, 10])
